In [170]:
import random
import pandas as pd
import yaml
import joblib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from modeling import ThreeLayerClassifier
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

team_rb_stats =  pd.read_csv('data/team_rb_stats.csv',index_col=0)
team_qb_stats= pd.read_csv('data/team_qb_stats.csv',index_col=0)
team_receiver_stats = pd.read_csv('data/team_receiver_stats.csv',index_col=0)
team_stats = pd.read_csv('data/agg/team_stats.csv',index_col=0).rename({'recent_team':'team_name'},axis=1)
opp_stats = pd.read_csv('data/agg/opp_stats.csv',index_col=0).rename({'opponent_team':'team_name'},axis=1)

In [171]:
players = pd.read_parquet('data/agg/player_weekly_agg.parquet')


#players = player_stats.merge(players,left_on=['player_id','week','season'],right_on=['gsis_id','week','season'])

In [172]:
players.loc[(players.player_id=='00-0036322') & (players.season==2024)].iloc[-3]
# will ship '00-0039746'

season                                                                           2024
team_name                                                                         MIN
week                                                                             10.0
game_type                                                                         REG
depth_team                                                                        1.0
last_name                                                                   Jefferson
first_name                                                                     Justin
football_name                                                                  Justin
formation                                                                     Offense
gsis_id                                                                    00-0036322
jersey_number                                                                    18.0
position                                              

In [173]:
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']

In [174]:
class Player():
    def __init__(self, d):
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.depth_team= int(d['dense_depth'])
        self.stats = {x:0 for x in stat_cols}
        self.stats['air_yards'] = 0
        self.stats['yac'] = 0
        self.features= d.to_dict()

    def show_headshot(self):
        url = self.headshot_url
        response = requests.get(url)
        response.raise_for_status()  # Ensure the request was successful
        img = Image.open(BytesIO(response.content))
        plt.imshow(img)
        plt.axis('off')  # Turn off axis for cleaner display
        plt.show()

    def __getattr__(self, name):
            # Redirect attribute access to the stats dictionary
            if name in self.stats:
                return self.stats[name]
            elif name in self.features:
                return self.features[name]
            raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    


In [175]:
class QB(Player):
    def __init__(self, d):  # noqa: F811
        super().__init__(d)
        self.name = d["full_name"]
        self.features= d.to_dict()

    def __repr__(self):
        return f"QB:{self.name} has {self.completions} completions for {self.passing_yards} yards"


class RB(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]
        self.features= d.to_dict()

    def __repr__(self):
        return f"RB:{self.name} has {self.carries} carries for {self.rushing_yards} yards"


class WR(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]


    def __repr__(self):
        return f"WR:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"
    
class TE(Player):
    def __init__(self, d):
        super().__init__(d)
        self.name = d["full_name"]
        self.id = d["gsis_id"]


    def __repr__(self):
        return f"TE:{self.name} has {self.receptions} receptions for {self.receiving_yards} yards"


class Team:
    def __init__(self, name: str, season: int, week: int, use_current_injuries=False):
        self.name = name
        self.score = 0
        self.plays = 0
        self.features = {
            'last_rusher_drive':-1,
            'last_rusher_team':-1
        }
        self.team_stats = fetch_row_or_latest(team_stats, self.name, season, week)
        self.opp_stats = fetch_row_or_latest(opp_stats, self.name, season, week)
        self.players = players.loc[
            (players.team_name == name) & (players.season == season)
        ]
        self.players = self.players.loc[
            (self.players.week == min(self.players.week.max(), week))
            & (self.players.formation == "Offense")
            & (self.players.position.isin(["QB", "WR", "TE", "RB"]))
        ].sort_values(by='dense_depth')

        self.QBs = self.get_players_by_position("QB")
        self.RBs = self.get_players_by_position("RB")
        self.WRs = self.get_players_by_position("WR")
        self.TEs = self.get_players_by_position("TE")
        self.rb_stats = fetch_row_or_latest(team_rb_stats, self.name, season, week)

        self.team_receiver_stats = fetch_row_or_latest(team_receiver_stats, self.name, season, week)

    def get_players_by_position(self, position: str):
        """Filter players by position and create player objects."""
        position_data = self.players[(self.players["position"] == position)].fillna(0)

        # Create player objects based on position
        players = []
        for _, player_data in position_data.iterrows():
            if position == "WR":
                players.append(WR(player_data))
            elif position == "RB":
                players.append(RB(player_data))
            elif position == "QB":
                players.append(QB(player_data))
            elif position == "TE":
                players.append(TE(player_data))
        return players
    
    def get_depth_pos(self, pos: str, depth: int):
        '''input a position and team depth, to get the player
            used to go from ML output -> player object'''
        if pos == 'WR':
            for player in self.WRs:
                if player.depth_team == depth:
                    return player
        if pos == 'RB':
            for player in self.RBs:
                if player.depth_team == depth:
                    return player
        if pos == 'TE':
            for player in self.TEs:
                if player.depth_team == depth:
                    return player
        raise ValueError('You want a player that does not exist')
    
    def fetch_row_or_latest(df, team, season, week):
        df = df.loc[
            (df.team_name == team) & (df.season == season)
        ]
        row = df.loc[
            (df.week == min(df.week.max(), week))
        ].to_dict(orient='records')[0]
        return row

    

    def __repr__(self):
        return f"{self.name} has {self.score} points"

In [176]:
team_stats.columns

Index(['team_name', 'season', 'week', 'completions_roll', 'attempts_roll',
       'carries_roll', 'passing_yards_roll', 'rushing_yards_roll',
       'pass_pct_roll'],
      dtype='object')

In [177]:
vik = Team('MIN',2024, 13)
det = Team('DET',2024, 13)
vik.team_stats

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_27256\2098100949.py:73: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  position_data = self.players[(self.players["position"] == position)].fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_27256\2098100949.py:73: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  position_data = self.players[(self.players["position"] == position)].fillna(0)


{'team_name': 'MIN',
 'season': 2024,
 'week': 11,
 'completions_roll': 21.142857142857142,
 'attempts_roll': 31.0,
 'carries_roll': 30.714285714285715,
 'passing_yards_roll': 253.7142857142857,
 'rushing_yards_roll': 112.71428571428572,
 'pass_pct_roll': 0.5065697690827448}

In [178]:
class GameState:


	def __init__(self, home, away, config):
		self.home =home
		self.away = away
		self.quarter = 1
		self.possession = None
		self.defending = None
		self.down = 1
		self.ydstogo = 10
		self.ball_position = 65  # Yardline (0-100), 0 is score, 100 is safety
		self.clock = 900  # Seconds in the current quarter (15 mins = 900 seconds)
		self.drive = 0
		self.home_timeouts = 3
		self.away_timeouts = 3
		self.pbp = []
		self.player = None
		self.run_or_pass = joblib.load('models/run_or_pass.joblib')
		self.run_or_pass_cols = config['run_or_pass_cols']
		self.choose_rusher = joblib.load('models/choose_rusher.joblib')
		self.choose_rusher_cols = config['choose_rusher_cols']
		self.choose_receiver = joblib.load('models/choose_receiver.joblib')
		self.choose_receiver_cols = config['choose_receiver_cols']
		self.air_yards_cols = config['air_yards_cols']
		self.receiver_idx_to_pos = config['receiver_idx_to_pos']
		self.rush_yard_cols = config['rush_yard_cols']
		self.complete_pass_cols = config['complete_pass_cols']
		self.complete_pass_model = joblib.load('models/complete_pass.joblib')
		self.play_encoding = config['play_encoding']
		self.wind = random.randint(0,10)
		self.temp = random.randint(40,90)
		self.play_functions = {
			'field_goal': self.field_goal,
			'no_play': self.run_play,
			'pass': self.pass_play,
			'punt': self.punt,
			'qb_kneel': self.qb_kneel,
			'qb_spike': self.qb_spike,
			'run': self.run_play
			}
		
		
	def switch_poss(self):
		self.possession.features['last_rusher_drive'] = -1
		self.possession = self.away if self.possession == self.home else self.home 
		self.defending = self.away if self.defending == self.home else self.home 
		self.ball_position = 100 - min(self.ball_position, 99)
		self.down = 1
		self.ydstogo = min(10, self.ball_position)
		self.drive += 1
		return
	
	def kickoff(self):
		self.switch_poss()
		self.ball_position = 65
		pass
	
	def start_game(self):
		lost_kickoff = random.choice((self.home, self.away))
		self.possession=lost_kickoff
		self.kickoff()
		print(f'{self.possession.name} has won the kickoff')
		self.log_play('kickoff',0)

	def play(self, team):
		team.plays += 1
		raw_features = self.pbp[-1]
		raw_features.update(team.team_stats)
		raw_features.update(self.defending.opp_stats)
		features = [raw_features[key] for key in self.run_or_pass_cols]
		preds = self.run_or_pass.predict_proba([features])
		play_type_int = np.random.choice(len(preds[0]), p=preds[0])
		play_type = self.play_encoding.get(play_type_int, 1)
		yds = self.play_functions[play_type](team)
		self.log_play(play_type, yds)
		self.ydstogo -= yds
		self.ball_position -= yds
		self.clock -= random.randint(10, 40)
		self.td_check(team)
		self.check_downs(team)


	def log_play(self,play_type, yds,verbose=False):
		"""Logs the context of the game state at each play."""
		play_data = {
			"possession": self.possession.name,
			"quarter": self.quarter,
			"down": self.down,
			"ydstogo": self.ydstogo,
			"goal_to_go": int(self.ball_position < 10),
			"yardline_100": self.ball_position,
			"total_home_score": self.home.score,
			"total_away_score": self.away.score,
			"posteam_score": self.possession.score,
			"defteam_score": self.home.score if self.possession == self.away else self.away.score,
			"score_differential": (self.home.score - self.away.score),
			"wind": self.wind,
			"temp": self.temp,
			"quarter_seconds_remaining": self.clock,
			'half_seconds_remaining': self.clock + (900 * (self.quarter%2)),
    		'game_seconds_remaining': self.clock + (900 * (4-self.quarter)),
			'drive':self.drive,
			'spread_line':-2.5,
			'total_line': 42.5,
			'play_type': play_type,
			'yards_gained': yds,
			'player': self.player
		}
		if verbose:
			print(
				f'{self.possession.name} {play_type} for {yds} yards, {self.pbp[-1]['yardline_100']} yd line,' + 
				f' {self.pbp[-1]['ydstogo']} yds to go on {self.pbp[-1]['down']} down.' + 
				f' {self.pbp[-1]['quarter_seconds_remaining'] // 60}:{self.pbp[-1]['quarter_seconds_remaining']  % 60} left'
			)
		self.pbp.append(play_data)


	
	def sample_run_yards(self, model, player):
		#rush_yards_cols = self.rush_yards_cols
		raw_features = dict(self.pbp[-1])
		raw_features.update(player.features)
		x = [raw_features[key] for key in self.rush_yard_cols]
		x = torch.tensor(x)
		with torch.no_grad():
			preds = model(x.reshape(1,-1))[0]
			preds = torch.softmax(preds,0)
		sample = (torch.multinomial(preds,1)).item() - 5
		return min((sample, raw_features['yardline_100']))

	def sample_air_and_yac(self, air_model, yac_model, player):
		raw_features = dict(self.pbp[-1])
		raw_features.update(player.features)
		x = [raw_features[key] for key in self.air_yards_cols]
		x = torch.tensor(x)
		with torch.no_grad():
			preds = air_model(x.reshape(1,-1))[0]
			preds = torch.softmax(preds,0)
		air_yards = (torch.multinomial(preds,1)).item() - 20
		if air_yards >= self.ball_position: #touchdown at catch
			return air_yards, 0
		
		x = torch.cat((x, torch.tensor([air_yards])))
		with torch.no_grad():
			preds = yac_model(x.reshape(1,-1))[0]
			preds = torch.softmax(preds,0)
		yac = (torch.multinomial(preds,1)).item() - 25
		return air_yards, min(yac, (self.ball_position - air_yards))
	
	def sample_completion(self, qb, receiver, air_yards):
		raw_features = dict(self.pbp[-1])
		raw_features.update(receiver.features)
		raw_features['air_yards'] = air_yards
		qb_features = {(key+'_qb'):value for key,value in qb.features.items()}
		raw_features.update(qb_features)
		features = [raw_features[key] for key in self.complete_pass_cols]
		preds = self.complete_pass_model.predict_proba([features])
		receiver = np.random.choice(len(preds[0]), p=preds[0])
		return  np.random.choice(len(preds[0]), p=preds[0])
		
		

	def run_play(self, team):
		raw_features = dict(self.pbp[-1])
		raw_features.update(team.rb_stats)
		raw_features.update(team.features)
		raw_features.update(team.features)
		features = [raw_features[key] for key in self.choose_rusher_cols]
		preds = self.choose_rusher.predict_proba([features])
		rusher_depth = np.random.choice(len(preds[0]), p=preds[0])
		try:
			player = team.get_depth_pos('RB', int(rusher_depth))
		except:
			player = team.RBs[0]
		player.carries += 1
		yds = self.sample_run_yards(run_yards_model, player)
		player.rushing_yards += yds
		self.player = player.name
		team.features['last_rusher_team'] = rusher_depth
		team.features['last_rusher_drive'] = rusher_depth
		return yds
	




	def pass_play(self, team):
		passer = team.QBs[0]
		raw_features = dict(self.pbp[-1])
		raw_features.update(team.team_receiver_stats)
		raw_features.update(team.features)
		features = [raw_features[key] for key in self.choose_receiver_cols]
		preds = self.choose_receiver.predict_proba([features])
		receiver = np.random.choice(len(preds[0]), p=preds[0])
		pos, depth = self.receiver_idx_to_pos[receiver].split('_')
		receiver = team.get_depth_pos(pos, int(depth))
		passer.attempts += 1
		receiver.targets += 1
		air_yards, yac = self.sample_air_and_yac(air_yards_model,yac_model, receiver)
		qb_features = {}
		qb_features.update(passer.features)

		if self.sample_completion(passer, receiver, air_yards):
			passer.completions += 1
			receiver.receptions += 1
			yds = air_yards + yac
			receiver.air_yards += air_yards
			receiver.yac += yac
			receiver.receiving_yards += yds
			passer.passing_yards += yds
		else:
			yds = 0
		self.player = receiver.name
		return yds
	
	
	def punt(self, team):
		self.switch_poss()
		self.ball_position += random.randint(30, 60)
		if self.ball_position >= 100:
			self.ball_position = 20
		self.player=None
		return 0
	
	def field_goal(self,team):
		result = random.randint(0,100)
		if result > (2 * self.ball_position):
			team.score += 3
			self.switch_poss()
			self.ball_position = 65
			print(f'{team.name} scored a FG')
		else:
			print(f'{team.name} missed FG')
			self.switch_poss()
		self.player=None
		return 0
	
	def qb_kneel(self, team):
		# Implementation of qb kneel play
		print("QB kneel executed.")
		return -1

	def qb_spike(self, team):
		# Implementation of qb spike play
		print("QB spike executed.")
		return 0
	
	def td_check(self, team):
		if self.ball_position <= 0:
			team.score += 7
			self.kickoff()
			print(f'{team.name} scored a TD')
		return
	
	def check_downs(self, team):
		if self.ydstogo <= 0:
			self.ydstogo = 10
			self.down = 1
		elif self.down == 4:
			self.switch_poss()
		else:
			self.down += 1


	def play_quarter(self):
		self.clock = 900
		while self.clock > 0:
			self.play(self.possession)
		self.quarter += 1
		print(f'{self.home.name}:{self.home.score}')
		print(f'{self.away.name}:{self.away.score}')

	def play_game(self):
		while self.quarter <= 4:
			self.play_quarter()
			print(self.quarter)


In [179]:
vik.team_stats

{'team_name': 'MIN',
 'season': 2024,
 'week': 11,
 'completions_roll': 21.142857142857142,
 'attempts_roll': 31.0,
 'carries_roll': 30.714285714285715,
 'passing_yards_roll': 253.7142857142857,
 'rushing_yards_roll': 112.71428571428572,
 'pass_pct_roll': 0.5065697690827448}

In [180]:
with open('models/feature_config.yaml', 'r') as file:
    config = yaml.safe_load(file)

vik = Team('MIN',2024, 11)
det = Team('DET',2024, 11)
game = GameState(vik, det, config)
game.start_game()
game.play_game()


DET has won the kickoff
DET scored a TD
DET scored a TD
MIN missed FG
DET scored a TD
DET missed FG
MIN scored a TD
MIN:7
DET:21
2
DET scored a TD
DET scored a FG
MIN:7
DET:31
3
MIN missed FG
DET scored a TD
MIN scored a TD
DET scored a TD
MIN scored a TD
DET scored a TD
MIN:21
DET:52
4
DET scored a TD
MIN scored a TD
DET scored a TD
MIN:28
DET:66
5


In [181]:
det.rb_stats

{'team_name': 'DET',
 'season': 2024,
 'week': 11.0,
 'game_type': 'REG',
 'gsis_id_RB_1': '00-0039139',
 'gsis_id_RB_2': '00-0035567',
 'gsis_id_RB_0': '00-0035685',
 'full_name_RB_1': 'Jahmyr Gibbs',
 'full_name_RB_2': 'Craig Reynolds',
 'full_name_RB_0': 'David Montgomery',
 'carries_roll_RB_1': 13.285641125449574,
 'carries_roll_RB_2': 1.6365150897701912,
 'carries_roll_RB_0': 13.384855897592905,
 'rushing_yards_roll_RB_1': 78.55150826301991,
 'rushing_yards_roll_RB_2': 10.080946776963938,
 'rushing_yards_roll_RB_0': 56.75730133792503,
 'rushing_tds_roll_RB_1': 0.801500538546599,
 'rushing_tds_roll_RB_2': 0.0049019605602175,
 'rushing_tds_roll_RB_0': 1.1196713712427848,
 'rushing_first_downs_roll_RB_1': 3.0316371236506914,
 'rushing_first_downs_roll_RB_2': 0.4646510309916307,
 'rushing_first_downs_roll_RB_0': 4.5374414551106375,
 'receptions_roll_RB_1': 1.6675041856510162,
 'receptions_roll_RB_2': 0.0001784167510769,
 'receptions_roll_RB_0': 2.155831147526209,
 'targets_roll_RB_1':

In [182]:
for i in range(10):
    game = GameState(vik, det, config)
    game.start_game()
    game.play_game()
    game.home.score=0
    game.away.score=0

MIN has won the kickoff
MIN scored a TD
MIN missed FG
DET scored a TD
MIN scored a TD
MIN:42
DET:73
2
DET scored a TD
MIN scored a TD
MIN scored a TD
MIN:56
DET:80
3
MIN missed FG
DET scored a TD
MIN:56
DET:87
4
MIN scored a TD
MIN scored a TD
DET scored a TD
MIN:70
DET:94
5
DET has won the kickoff
DET scored a TD
MIN scored a TD
DET scored a TD
DET scored a FG
MIN scored a TD
MIN:14
DET:17
2
MIN scored a TD
DET scored a TD
DET missed FG
MIN scored a TD
MIN missed FG
DET scored a TD
MIN:28
DET:31
3
DET scored a TD
MIN scored a TD
MIN scored a FG
MIN:38
DET:38
4
DET scored a TD
MIN scored a TD
QB kneel executed.
QB kneel executed.
QB kneel executed.
QB kneel executed.
MIN:45
DET:45
5
DET has won the kickoff
DET scored a TD
MIN missed FG
DET scored a TD
MIN:0
DET:14
2
DET missed FG
MIN scored a TD
DET scored a TD
DET scored a TD
MIN:7
DET:28
3
MIN scored a TD
DET scored a TD
MIN scored a FG
MIN missed FG
MIN:17
DET:35
4
MIN missed FG
DET scored a TD
DET scored a TD
MIN scored a TD
MIN:24

In [183]:
det.WRs + det.TEs

[WR:Amon-Ra St. Brown has 131 receptions for 1238 yards,
 WR:Tim Patrick has 15 receptions for 114 yards,
 WR:Kalif Raymond has 35 receptions for 457 yards,
 WR:Allen Robinson has 8 receptions for 70 yards,
 TE:Sam LaPorta has 35 receptions for 491 yards,
 TE:Brock Wright has 8 receptions for 37 yards]

In [184]:
1041/10

104.1

In [185]:
pbp = pd.json_normalize(game.pbp)
pbp

,possession,quarter,down,ydstogo,goal_to_go,yardline_100,total_home_score,total_away_score,posteam_score,defteam_score,...,carries_roll,passing_yards_roll,rushing_yards_roll,pass_pct_roll,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,pass_pct_opp_roll
0,MIN,1,1,10,0,65,0,0,0,0,...,30.714286,253.714286,112.714286,0.506570,22.142857,35.285714,21.142857,253.857143,75.714286,0.612389
1,MIN,1,1,10,0,65,0,0,0,0,...,30.714286,253.714286,112.714286,0.506570,22.142857,35.285714,21.142857,253.857143,75.714286,0.612389
2,MIN,1,2,9,0,64,0,0,0,0,...,30.714286,253.714286,112.714286,0.506570,22.142857,35.285714,21.142857,253.857143,75.714286,0.612389
3,MIN,1,3,8,0,63,0,0,0,0,...,30.714286,253.714286,112.714286,0.506570,22.142857,35.285714,21.142857,253.857143,75.714286,0.612389
4,MIN,1,1,10,0,55,0,0,0,0,...,30.714286,253.714286,112.714286,0.506570,22.142857,35.285714,21.142857,253.857143,75.714286,0.612389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,MIN,4,3,4,0,59,38,49,38,49,...,30.714286,253.714286,112.714286,0.506570,22.142857,35.285714,21.142857,253.857143,75.714286,0.612389
143,MIN,4,1,10,0,36,38,49,38,49,...,30.714286,253.714286,112.714286,0.506570,22.142857,35.285714,21.142857,253.857143,75.714286,0.612389
144,MIN,4,1,10,0,15,38,49,38,49,...,31.714286,263.000000,147.571429,0.440135,23.714286,38.000000,22.428571,255.285714,102.571429,0.624827
145,DET,4,2,10,0,65,45,49,49,45,...,31.714286,263.000000,147.571429,0.440135,23.714286,38.000000,22.428571,255.285714,102.571429,0.624827
